this notebook tries to visualize some models examined in `basic_for_certain_configs_cc2normed.ipynb`. I will check those with biggest ccnorm2 difference between unroll=1 and unroll=5. I will ignore the middle ones, and see hope to see some visible differences.

the models are exported using `scripts/model_export/yuanyuan_8k_a_3day/maskcnn_polished_with_local_pcn/export_certain_configs.py` and verified with `scripts/model_export/yuanyuan_8k_a_3day/maskcnn_polished_with_local_pcn/export_certain_configs_check_keras.py`.

the visualization follows `https://github.com/leelabcnbc/tang_jcompneuro_revision/blob/master/results_ipynb/plots/main/demo_neuron_vis.ipynb`

In [1]:
from sys import path
from copy import deepcopy
from collections import defaultdict
from os.path import join, exists
from thesis_v2 import dir_dict
import numpy as np
folder_to_check = 'scripts/training/yuanyuan_8k_a_3day/maskcnn_polished_with_local_pcn'
path.insert(0, join(dir_dict['root'], folder_to_check))
from submit_certain_configs import param_iterator_obj
from key_utils import keygen, script_keygen

In [2]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial


cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [3]:
# then let's check the avg(ccnorm2 at unroll=5) - avg(ccnorm2 at unroll=1) for all cases.

In [4]:
from thesis_v2.training.training_aux import load_training_results

constant_params = {
    'out_channel': 16,
    'num_layer': 2,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'split_seed': 'legacy',
    'smoothness_name': '0.000005',
    'scale_name': '0.01',
    'pcn_no_act': False,
    'pcn_bias': True,
    'pcn_bypass': False,
    'bn_after_fc': False,
}

def do_one_case(param, dict_ret):
    for k1, v1 in constant_params.items():
        assert param[k1] == v1
    

    # assert param['model_seed'] == 0

    key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
    
    params_variable = {
        k: v for k, v in param.items() if k not in (constant_params.keys() | {'scale', 'smoothness'})
    }
    # 128 (configs; 7**2) * 3 (seeds) * 6 (number of cycles from 0-5)
    assert len(params_variable) == 9
#     print(params_variable)
    
    key_for_result_dict = frozenset({(k,v) for k, v in params_variable.items() if k not in {'model_seed', 'pcn_cls'}})
    assert len(key_for_result_dict) == 7
    
    # load the performance.
    result = load_training_results(key, return_model=False)
    
    if key_for_result_dict not in dict_ret:
        dict_ret[key_for_result_dict] = defaultdict(list)
        
    # calculate ccnorm^2.
    cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
    assert cc_raw.shape == (79,)
    ccnorm2 = ((cc_raw/cc_max_all_neurons)**2).mean()
    assert(np.isfinite(ccnorm2))
    
    dict_ret[key_for_result_dict][params_variable['pcn_cls']].append(ccnorm2)

In [5]:
def collect_data():
    dict_ret = dict()
    model_count = len(list(param_iterator_obj.generate()))
    # this should be 2304, which is 128 (configs) * 3 (seeds) * 6 (number of cycles from 0-5)
    assert model_count == 2304
    for idx, param_dict in enumerate(param_iterator_obj.generate()):
        if idx % 100 == 0:
            print(f'model {idx}/{model_count}')
        do_one_case(param_dict, dict_ret)
    return dict_ret

In [6]:
dict_all_ccnorm2 = collect_data()

model 0/2304
model 100/2304
model 200/2304
model 300/2304
model 400/2304
model 500/2304
model 600/2304
model 700/2304
model 800/2304
model 900/2304
model 1000/2304
model 1100/2304
model 1200/2304
model 1300/2304
model 1400/2304
model 1500/2304
model 1600/2304
model 1700/2304
model 1800/2304
model 1900/2304
model 2000/2304
model 2100/2304
model 2200/2304
model 2300/2304


In [7]:
def show_diff_every_key(dict_ret):
    for key, value in dict_ret.items():
        if np.mean(value[5])-np.mean(value[1]) >= 0.02:
            print(tuple(key), np.mean(value[5])-np.mean(value[1]), np.mean(value[5])-np.mean(value[0]),
                 np.mean(value[5]), np.mean(value[0]))
show_diff_every_key(dict_all_ccnorm2)

(('act_fn', 'relu'), ('input_size', 100), ('loss_type', 'mse'), ('pcn_final_act', True), ('bn_before_act', True), ('pcn_bn', False), ('pcn_bn_post', False)) 0.027412399616660088 -0.016090200536684918 0.3849583976488696 0.4010485981855545
(('act_fn', 'relu'), ('input_size', 100), ('loss_type', 'mse'), ('bn_before_act', True), ('pcn_final_act', False), ('pcn_bn', False), ('pcn_bn_post', False)) 0.03168640151885299 -0.028440148572544954 0.37292742773691545 0.4013675763094604
(('act_fn', 'relu'), ('pcn_final_act', True), ('bn_before_act', True), ('loss_type', 'poisson'), ('pcn_bn', False), ('pcn_bn_post', False), ('input_size', 50)) 0.062192652728396414 0.03907294184369914 0.5513841263769397 0.5123111845332405
(('act_fn', 'relu'), ('loss_type', 'poisson'), ('pcn_bn', True), ('pcn_final_act', False), ('bn_before_act', False), ('pcn_bn_post', False), ('input_size', 50)) 0.023366283075041405 0.013141693432625612 0.6190519431220595 0.6059102496894339
(('act_fn', 'relu'), ('loss_type', 'poisson

In [8]:
# in these cases, the differences look big enough.
candidates_to_visualize = {
    # 0.062192652728396414 0.03907294184369914 0.5513841263769397 0.5123111845332405
    (('loss_type', 'poisson'), ('input_size', 50), ('pcn_bn', False), ('pcn_final_act', True), ('pcn_bn_post', False), ('act_fn', 'relu'), ('bn_before_act', True)),
    # 0.0629323663988261 0.016065903548365235 0.5353156029314999 0.5192496993831347
    (('pcn_final_act', False), ('input_size', 50), ('pcn_bn', False), ('loss_type', 'mse'), ('pcn_bn_post', False), ('bn_before_act', True), ('act_fn', 'softplus')),
    # 0.04289359964839312 0.017826213557316284 0.581971909255969 0.5641456956986527
    (('loss_type', 'poisson'), ('pcn_final_act', False), ('input_size', 50), ('pcn_bn', False), ('pcn_bn_post', False), ('bn_before_act', True), ('act_fn', 'softplus')),
    # 0.026568696278369885 0.034808706083833796 0.6656576629735268 0.630848956889693
    (('loss_type', 'poisson'), ('pcn_bn', False), ('pcn_final_act', True), ('input_size', 100), ('bn_before_act', True), ('pcn_bn_post', True), ('act_fn', 'softplus')),
}



In [14]:
def visualize_one_case(key_for_result_dict):
    # show cls=0, 1, 5
    print(tuple(key_for_result_dict))
    for cls_to_check in (0, 1, 5):
        # for each one, show model_seed=0,1,2
        for model_seed in range(3):
            param_dict = {
                **constant_params, **{k: v for (k, v) in key_for_result_dict},
                **{'pcn_cls': cls_to_check, 'model_seed': model_seed}
            }
            print(f'cls {cls_to_check}, seed {model_seed}')
            
            # do the visualization.
            

def visualize_all_cases():
    for cand in candidates_to_visualize:
        visualize_one_case(frozenset(cand))
        
visualize_all_cases()

(('pcn_final_act', False), ('pcn_bn', False), ('pcn_bn_post', False), ('loss_type', 'poisson'), ('act_fn', 'softplus'), ('bn_before_act', True), ('input_size', 50))
cls 0, seed 0
cls 0, seed 1
cls 0, seed 2
cls 1, seed 0
cls 1, seed 1
cls 1, seed 2
cls 5, seed 0
cls 5, seed 1
cls 5, seed 2
(('act_fn', 'relu'), ('pcn_final_act', True), ('pcn_bn', False), ('pcn_bn_post', False), ('loss_type', 'poisson'), ('bn_before_act', True), ('input_size', 50))
cls 0, seed 0
cls 0, seed 1
cls 0, seed 2
cls 1, seed 0
cls 1, seed 1
cls 1, seed 2
cls 5, seed 0
cls 5, seed 1
cls 5, seed 2
(('pcn_final_act', False), ('pcn_bn', False), ('pcn_bn_post', False), ('loss_type', 'mse'), ('act_fn', 'softplus'), ('bn_before_act', True), ('input_size', 50))
cls 0, seed 0
cls 0, seed 1
cls 0, seed 2
cls 1, seed 0
cls 1, seed 1
cls 1, seed 2
cls 5, seed 0
cls 5, seed 1
cls 5, seed 2
(('pcn_final_act', True), ('pcn_bn', False), ('pcn_bn_post', True), ('input_size', 100), ('loss_type', 'poisson'), ('act_fn', 'softplus'